In [ ]:
import pandas as pd
import polars as pl
import numpy as np

import datetime as dt
import os

import itertools as itl

import sys

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from py_functions.MEAS_recordings import retrieve_peaks

## Running cells

In [ ]:
from nb_vars import FOLDER_INPUT_BASE, FOLDER_INTERMEDIATE

**EXP1**

In [ ]:
folder_input = f'{FOLDER_INPUT_BASE}'
folder_output = f'{FOLDER_INTERMEDIATE}'

os.makedirs(folder_output, exist_ok=True)

dict_condition_wellname = {'HEK PLKO': ['A2', 'B6', 'C1', 'D1'], 
                           'HEK 219': ['A5', 'B2', 'C3', 'D6'],
                           'ASTRO PLKO': ['A1', 'A3', 'C6', 'D2'],
                           'ASTRO 219': ['A4', 'A6', 'B1', 'D4'],
                           'BLANCO': ['C4', 'D5'], }

dict_condition_wellnumber = {'HEK PLKO': ['12', '26', '31', '41'], 
                           'HEK 219': ['15', '22', '33', '46'],
                           'ASTRO PLKO': ['11', '13', '36', '42'],
                           'ASTRO 219': ['14', '16', '21', '24'],
                           'BLANCO': ['34', '45'], }

dict_condition_replicates = {'HEK PLKO': [1, 2, 3, 4], 
                           'HEK 219': [1, 2, 3, 4],
                           'ASTRO PLKO': [1, 2, 3, 4],
                           'ASTRO 219': [1, 2, 3, 4],
                           'BLANCO': [1, 2], }

dict_times_dirs = {"Base": "iPSCs_DIV20_250uA_PRE.TRATAMIENTO(000)"} | {f"{i * 10} min": f"iPSCs_DIV20_250uA_TRATAMIENTO(0{i:0>2})" for i in range(18)}

conditions = ['HEK PLKO', 'HEK 219', 'ASTRO PLKO', 'ASTRO 219', 'BLANCO']
times = ["Base"] + [f"{i * 10} min" for i in range(18)]

list_conditions = [#('Condition', 'Time exp', 'Wells', 'Well_num', 'Replicate', 'Folder') 
                    (cond, time, dict_condition_wellname[cond], dict_condition_wellnumber[cond], dict_condition_replicates[cond], dict_times_dirs[time])
                    for cond, time in itl.product(conditions, times)
                   ]

In [ ]:
print(f'OUTPUT: {folder_output}/df_peaks_full.csv')

list_df_peaks = []

for condition, time_exp, list_wells, list_wells_num, replicates, sample_base in list_conditions:
    for well, well_num, replicate in zip(list_wells, list_wells_num, replicates):
        print(condition, time_exp, well, well_num, replicate, sample_base)
        list_peak_times, list_electrodes = retrieve_peaks(root=folder_input, sample_base=sample_base, well=(int(well_num[0]), int(well_num[1])))
        
        df_peaks_i = pl.DataFrame({'condition': [condition] * len(list_peak_times), 
                                    'time_exp': [time_exp] * len(list_peak_times), 
                                    'well': [well] * len(list_peak_times), 
                                    'well_num': [well_num] * len(list_peak_times), 
                                    'replicate': [replicate] * len(list_peak_times), 
                                    'electrode': list_electrodes, 
                                    'time': list_peak_times
                                    })
        
        list_df_peaks.append(df_peaks_i)

df_peaks_exp_1 = pl.concat(list_df_peaks)
df_peaks_exp_1.write_csv(f'{folder_output}/df_peaks_full.csv')


In [ ]:
!rm -rf $PWD/tmp